In [1]:
# Import des bibliothèques nécessaires
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import sys
# Ajout du chemin vers les fonctions personnalisées
sys.path.append('/home/onyxia/work/Depot_Git/scripts')
from my_function import compute_centroid, compute_avg_distance_to_centroid, create_bar_plot, create_violin_plot,get_polygon_labels
sys.path.append('/home/onyxia/work/libsigma')
import classification as cla

In [2]:
# Définition des paramètres 
my_folder = '/home/onyxia/work/Depot_Git/results/data'
in_vector = os.path.join(my_folder, 'sample/Sample_BD_foret_T31TCJ.shp')
raster_ndvi = os.path.join(my_folder, 'img_pretraitees/Serie_temp_S2_ndvi.tif')
diag_baton_dist_centroide_classe_path = os.path.join(my_folder, "../figure/diag_baton_dist_centroide_classe.png")
violin_plot_dist_centroide_by_poly_by_class_path = os.path.join(my_folder, "../figure/violin_plot_dist_centroide_by_poly_by_class.png")
out_image = os.path.splitext(in_vector)[0] + '_v2.tif'
field_name = 'Code'  # field containing the numeric label of the classes


In [3]:

# for those parameters, you know how to get theses information if you had to
sptial_resolution = 10
xmin = 501127.9696999999
ymin = 6240654.023599998
xmax = 609757.9696999999
ymax = 6314464.023599998

# define command pattern to fill with paremeters
cmd_pattern = ("gdal_rasterize -a {field_name} "
               "-tr {sptial_resolution} {sptial_resolution} "
               "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
               "{in_vector} {out_image}")

# fill the string with the parameter thanks to format function
cmd = cmd_pattern.format(in_vector=in_vector, xmin=xmin, ymin=ymin, xmax=xmax,
                         ymax=ymax, out_image=out_image, field_name=field_name,
                         sptial_resolution=sptial_resolution)

# execute the command in the terminal
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [4]:
# Extraction des échantillons sous forme d'un tableau Numpy 
sample_filename = os.path.join(my_folder, 'sample/Sample_BD_foret_T31TCJ_v2.tif')
image_filename = os.path.join(my_folder, 'img_pretraitees/Serie_temp_S2_ndvi.tif')
X, Y, t = cla.get_samples_from_roi(image_filename, sample_filename)
print(X.shape)
print(Y.shape)
unique_classes = np.unique(Y)
print("Classes uniques :", unique_classes)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


(4997108, 6)
(4997108, 1)
Classes uniques : [11 12 13 14 15 16 21 22 23 24 25 26 27 28 29]


### Calcul de distance à l'échelle de l'image entière 

In [5]:
List_classes = [11,12,13,14,15,23,24,25,26,28,29]
Y = Y.flatten() 
# Calculer la distance moyenne au centroïde pour chaque classe
class_avg_distances = {}
for cls in unique_classes:
    class_samples = X[Y == cls]  # Filtre les échantillons pour la classe actuelle
    centroid = compute_centroid(class_samples)
    avg_distance = compute_avg_distance_to_centroid(class_samples, centroid)
    class_avg_distances[cls] = avg_distance


### Graphique en bâton

In [6]:
create_bar_plot(class_avg_distances,diag_baton_dist_centroide_classe_path)

### Calcul à l'échelle de polygone

In [7]:
# Obtenez les étiquettes des polygones pour chaque pixel
polygon_labels = get_polygon_labels(Y)

# Dictionnaire pour stocker les distances moyennes par classe et par polygone
class_avg_distances_by_polygon = {}

# Calcul pour chaque classe
for cls in unique_classes:
    # Filtrer les labels des polygones pour correspondre aux classes qui nous intéressent 
    polygon_labels_filtered = polygon_labels[Y.flatten() == cls]
    polygon_samples = X[Y.flatten() == cls]  # Données pour une classe donnée
    
    # Calcul pour chaque polygone
    polygon_avg_distances = []
    for poly_id in np.unique(polygon_labels_filtered):
        # Sélectionner les échantillons du polygone courant
        poly_samples = polygon_samples[polygon_labels_filtered == poly_id]
        centroid = compute_centroid(poly_samples)
        avg_distance = compute_avg_distance_to_centroid(poly_samples, centroid)
        polygon_avg_distances.append(avg_distance)

    # Ajouter la moyenne des distances des polygones pour la classe courante
    class_avg_distances_by_polygon[cls] = polygon_avg_distances

### Violin plot

In [8]:

create_violin_plot(class_avg_distances_by_polygon,violin_plot_dist_centroide_by_poly_by_class_path )

In [9]:
print("Analyse terminée. Les graphiques sont enregistrés dans le dossier results/figure.")

Analyse terminée. Les graphiques sont enregistrés dans le dossier results/figure.
